In [22]:
# import packages
import glob
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import re
import linecache 
# Uncomment below line if miepython hasn't been install
%pip install miepython
import miepython
import warnings

Note: you may need to restart the kernel to use updated packages.


# Loading in Data
This data comes from the NASA Langley Research Center's Airborne Science Data for Atmopheric Composition [repository](https://www-air.larc.nasa.gov/). Aerosol data was recorded on NASA SARP flights from the [Langley Aerosol Research Group](https://science-data.larc.nasa.gov/large/).

### Creating One Dataframe
Relevant variables were merged using LaRC's Custom Data Merging Tool. Files are in .ict format but can still be read with pd.read_csv. The first lines in the .ict files are all the metadata and must be skipped.  

In [23]:
path = "raw_data/2019/merged_2019_SARP/"
all_files = glob.glob(path + "*.ict")
all_files.sort()
all_files

li = []
flight_number=['1','2','3']
index = 0
for filename in all_files:
    # column headers start on line 162 so we skip 161 rows
    df = pd.read_csv(filename, index_col=None, header=0, skiprows=161)
    df["flight number"] = [flight_number[index] for i in range(len(df))]
    li.append(df)
    index = index+1

all_raw = pd.concat(li, axis=0, ignore_index=True)

### FIREX-AQ Dataframe
In 2019 [FIREX-AQ](https://csl.noaa.gov/projects/firex-aq/) instruments were used on the 2019 SARP flights. Black carbon data is only available via the FIREX-AQ datasets. We load in the FIREX-AQ datasets from the SARP flights.

In [24]:
path = "raw_data/2019/merged_2019_FIREXAQ/"
all_files = glob.glob(path + "*.ict")
all_files.sort()
all_files

li = []
flight_number=['1','2','3']
index = 0
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, skiprows=187)
    df["flight number"] = [flight_number[index] for i in range(len(df))]
    li.append(df)
    index = index+1

all_raw_fire = pd.concat(li, axis=0, ignore_index=True)

### Dataframe Cleaning
Reduce SARP and FIREX-AQ dataframes to only include observations from both datasets.

In [25]:
all_raw_fire = all_raw_fire[all_raw_fire['Time_Start'].isin(all_raw['Time_Start'].unique())]
all_raw = all_raw[all_raw['Time_Start'].isin(all_raw_fire['Time_Start'].unique())]

### Adding Datetime Variables
LaRC data uses Day_Of_Year which must be converted to date. LaRC also has Time_Start and Time_Stop in UTC seconds which must be converted to h:m:s. The h:m:s start time and the date are combinded to create one datetime variable.

In [26]:
# for sarp

# add year
all_raw['year'] = 2019

# convert year and Day_Of_year to date
all_raw['date'] = pd.to_datetime(all_raw['year'] * 1000 + all_raw[' Day_Of_Year'], format='%Y%j')

# convert start and stop times to h:m:s format
all_raw['Time_Start hms'] = pd.to_datetime(all_raw['Time_Start'], unit='s').apply(lambda x: x.strftime("%H:%M:%S"))
all_raw['Time_Stop hms'] = pd.to_datetime(all_raw[' Time_Stop'], unit='s').apply(lambda x: x.strftime("%H:%M:%S"))

# combine h:m:s format and date to create datetime
all_raw['datetime_start'] = pd.to_datetime(all_raw['date'].astype(str) + ' ' + all_raw['Time_Start hms'].astype(str))


# for firexaq

# add year
all_raw_fire['year'] = 2019

# convert year and Day_Of_year to date
all_raw_fire['date'] = pd.to_datetime(all_raw_fire['year'] * 1000 + all_raw_fire[' Day_Of_Year'], format='%Y%j')

# convert start and stop times to h:m:s format
all_raw_fire['Time_Start hms'] = pd.to_datetime(all_raw_fire['Time_Start'], unit='s').apply(lambda x: x.strftime("%H:%M:%S"))
all_raw_fire['Time_Stop hms'] = pd.to_datetime(all_raw_fire[' Time_Stop'], unit='s').apply(lambda x: x.strftime("%H:%M:%S"))

# combine h:m:s format and date to create datetime
all_raw_fire['datetime_start'] = pd.to_datetime(all_raw_fire['date'].astype(str) + ' ' + all_raw_fire['Time_Start hms'].astype(str))

### Merge Datasets
Merge FIREX-AQ data and SARP data so the black carbon variable is now in the SARP dataset.

In [27]:
all_raw = pd.merge(all_raw,all_raw_fire[['datetime_start',' BC_mass_90_550_nm_SCHWARZ']],on='datetime_start', how='left')

### Data Cleaning
We only want to look at observations underneath the boundary layer which we set at 2000 feet. We want to remove all negative values and replace with NaN.

In [28]:
# remove entries under 2000 ft
df_new_under2k = all_raw[all_raw[' Pressure_Altitude'] <= 2000]

# replace -999999 and -888888 values with NaN
df_new_under2k = df_new_under2k.replace({-999999: np.nan})
df_new_under2k = df_new_under2k.replace({-888888: np.nan})

# checking for negative values
check = pd.DataFrame(df_new_under2k.min())

# removing negative values
df_new_under2k[df_new_under2k[[" NR_Chloride_PM1_AMS_JIMENEZ",
         " Seasalt_PM1_AMS_JIMENEZ",
         " MSA_PM1_AMS_JIMENEZ",
         " ClO4_PM1_AMS_JIMENEZ",
         " Bromine_PM1_AMS_JIMENEZ",
         " Iodine_PM1_AMS_JIMENEZ",
         " OSc_PM1_AMS_JIMENEZ",
         " f57_PM1_AMS_JIMENEZ",
         " f60_PM1_AMS_JIMENEZ",
         " f82_PM1_AMS_JIMENEZ",
         " f91_PM1_AMS_JIMENEZ",
         " fC4H9_PM1_AMS_JIMENEZ",
         " fC2H4O2_PM1_AMS_JIMENEZ",
         " fC7H7_PM1_AMS_JIMENEZ",
         " Abs532_stdPT_MOORE"]] < 0] = np.NaN


# Scientific Processing
We are using the Langley Aerosol Research Group data to estimate the refractive index and the size of the aerosols.

## Composition and Refractive Index
Organics, Ammonium Nitrate, Ammonium Sulfate, and Black Carbon all have different refractive indices. If we want to determine a total refractive index for the time interval, we must determine what fraction of the sample belongs to each aerosol. We will then use those weights and the aerosols' know refractive indices to create a weighted refractive index. 

### Aligning Units
Organics, Sulfate, Nitrate, and Ammonium are measured in µg/m<sup>3</sup>. Black Carbon is measured in ng/m<sup>3</sup>. We convert black carbon to µg/m<sup>3</sup> so the units align.

In [29]:
#BC_mass_90_550_nm_SCHWARZ in ng std m-3
#PM1_AMS_JIMENEZ in ug sm-3
#both are in standard cubic meters, BC is in nanograms and others are in micrograms

df_new_under2k[' BC_mass_90_550_nm_SCHWARZ'] = df_new_under2k[' BC_mass_90_550_nm_SCHWARZ']/1000

### Ammonium Nitrate and Ammonium Sulfate
Ammonium, Nitrate, and Sulfate ion concentrations are recorded instead of the concentrations of Ammonium Nitrate and Ammonium Sulfate. The following formulas are used to determine the concentrations based on the ions. We are using the molecular weight of sulfate ($mm_{SO_4}$) and the molecular weight of nitrate ($mm_{NO_3}$).

$ (NH_4)_2SO_4 = [SO_4 ^{2-}] + \frac{[SO_4^{2-}]\div mm_{SO_4} \times 2}{([SO_4 ^{2-}]\div mm_{SO_4} \times 2)+([NO_3 ^{-}]\div mm_{NO_3})} \times [NH_4^{+}] $ 

$ NH_4NO_3 = [NO_3 ^{-}] + \frac{[NO_3 ^{-}]\div mm_{NO_3}}{([SO_4 ^{2-}]\div mm_{SO_4} \times 2)+([NO_3 ^{-}]\div mm_{NO_3})} \times [NH_4^{+}] $ 

In [30]:
# molecular weight of sulfate is 96.07 g/mol
sul_mm = 96.07
# molecular wight of nitrate is 62.005 g/mol
nit_mm = 62.005

# calculate ammonium sulfate concentration
df_new_under2k['NH42SO4'] = df_new_under2k[' Sulfate_PM1_AMS_JIMENEZ'] + (((df_new_under2k[' Sulfate_PM1_AMS_JIMENEZ']/sul_mm)*2)/(((df_new_under2k[' Sulfate_PM1_AMS_JIMENEZ']/sul_mm)*2)+(df_new_under2k[' Nitrate_PM1_AMS_JIMENEZ']/nit_mm)))*df_new_under2k[' Ammonium_PM1_AMS_JIMENEZ']

# calculate ammonium nitrate concentration
df_new_under2k['NH4NO3'] = df_new_under2k[' Nitrate_PM1_AMS_JIMENEZ'] + ((df_new_under2k[' Nitrate_PM1_AMS_JIMENEZ']/nit_mm)/(((df_new_under2k[' Sulfate_PM1_AMS_JIMENEZ']/sul_mm)*2)+(df_new_under2k[' Nitrate_PM1_AMS_JIMENEZ']/nit_mm)))*df_new_under2k[' Ammonium_PM1_AMS_JIMENEZ']

# uncomment line below to test to make sure code works
# print(df_new_under2k[' Sulfate_PM1_AMS_JIMENEZ'] + df_new_under2k[' Nitrate_PM1_AMS_JIMENEZ'] + df_new_under2k[' Ammonium_PM1_AMS_JIMENEZ'])
# print( df_new_under2k['NH42SO4'] + df_new_under2k['NH4NO3'])

### Composition Fraction
The refractive index of the aerosol sample depends on what it is made of. We need to know what fraction of the sample is organics, ammonium nitrate, ammonium sulfate, and black carbon.

In [31]:
# summing up individual compositions to find the total composition
df_new_under2k["comp_sum"] = df_new_under2k[" OA_PM1_AMS_JIMENEZ"] + df_new_under2k["NH42SO4"] + df_new_under2k["NH4NO3"] + df_new_under2k[" BC_mass_90_550_nm_SCHWARZ"]

# determining the fraction of each aerosol
df_new_under2k["Org_frac"] = df_new_under2k[" OA_PM1_AMS_JIMENEZ"]/df_new_under2k["comp_sum"]
df_new_under2k["NH42SO4_frac"] = df_new_under2k["NH42SO4"]/df_new_under2k["comp_sum"]
df_new_under2k["NH4NO3_frac"] = df_new_under2k["NH4NO3"]/df_new_under2k["comp_sum"]
df_new_under2k["BC_frac"] = df_new_under2k[" BC_mass_90_550_nm_SCHWARZ"]/df_new_under2k["comp_sum"]

# check to make sure fraction code worked
# print(df_new_under2k["Org_frac"] + df_new_under2k["NH42SO4_frac"] + df_new_under2k["NH4NO3_frac"] + df_new_under2k["BC_frac"])

### Weighted Refractive Index
We will use the known refractive indices of Organics, Ammonium Nitrate, Ammonium Sulfate, and Black Carbon as well as our composition fractions to calculate a weighted refractive index. These refractive indices were used by [Langridge 2012](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2011JD017116). The refractive indices are complex numbers and therefore contain a real and imaginary part.

In [32]:
# real and imaginary part for organics
Org_rel = 1.63
Org_im = 0.02

# real and imaginary part for ammonium sulfate
NH42SO4_rel = 1.53
NH42SO4_im = 0

# real and imaginary part for ammonium nitrate
NH4NO3_rel = 1.6
NH4NO3_im = 0

# real and imaginary part for black carbon
BC_rel = 1.95
BC_im = 0.79

# weighted real and imaginary part
df_new_under2k['rel_av'] = df_new_under2k['Org_frac']*Org_rel + df_new_under2k['NH42SO4_frac']*NH42SO4_rel + df_new_under2k['NH4NO3_frac']*NH4NO3_rel + df_new_under2k['BC_frac']*BC_rel
df_new_under2k['im_av'] = df_new_under2k['Org_frac']*Org_im + df_new_under2k['NH42SO4_frac']*NH42SO4_im + df_new_under2k['NH4NO3_frac']*NH4NO3_im + df_new_under2k['BC_frac']*BC_im

## Aerosol Size
Aerosol size data is given by the concentration of aerosols within a certain diameter distribution bin. There are multiple bins within a dataset. Average diameter is determinded by summing up the aerosols across all bins and finding the weight of each bin. The weight is then multiplied by the diameter the bin represents to find a weighted diameter.

### Diameter Bins
Bin mid-point diameters were not in the merged metadata and has to be pulled from the raw LAS metadata.

In [33]:
# creates a list of bin columns
collist = df_new_under2k.filter(like='LAScold_bin').columns.tolist()

# pulls midpoint diameters from LAS metadata
# bin mid-point diameters are in um
sizes = linecache.getline("raw_data/2019/LAS_meta_SARP/SARP-LARGE-LAScold_DC8_20190716_R0.ict", 115) 

# split sizes list based on multiple delimeters
sizes = sizes.replace('OTHER_COMMENTS: Midpoint diameters of Bins 00 to 79 determined by DMA-classified ammonium sulfate calibrations are:', '')

# Split sizes list based on multiple delimeters
sizes = re.split(',|\n|\t', sizes)

# Remove the empty elements from the list
sizes = list(filter(None, sizes))

# Convert element in the list from type string to type float
sizes = [float(i) for i in sizes]

# creates dataframe of bin mid-points and column names
LAS = pd.DataFrame(
    {'columns': collist,
     'size': sizes
    })

# Midpoint diameters are in um, need in nm
LAS['size'] = LAS['size']*1000

# bin sizes range from year to year
# LAS from 2016 has 100nm to 5011.9nm
# LAS from 2019 has 100nm to 4796nm
# UHSAS from 2024 has 60.9nm to 985.9nm

# want range 100nm to 985.9nm
LAS = LAS[(LAS['size'] >= 100) & (LAS['size'] <= 985.9) ]

# turn sizes into an array
size_array = np.array(LAS['size'])

### Weighted Diameter
Find sum of all aerosols for each observation and find the weight of each bin. Use weight and bin mid-point diameter to find weighted diameter.

In [34]:
# sum aerosols across all bins
df_new_under2k['LAS_sum'] = df_new_under2k[LAS['columns']].sum(axis=1)

# create a new dataframe where each bin value is divided by the sum of all bins
weighted_df = df_new_under2k[LAS['columns']].div(df_new_under2k['LAS_sum'], axis=0)

# multiply each weight by the corresponding bin diameter
for i in range(len(LAS['columns'])):
   weighted_df[weighted_df.columns[i]] = weighted_df[weighted_df.columns[i]]*size_array[i]

# sum diameters
df_new_under2k['LAS_weighted'] = weighted_df[LAS['columns']].sum(axis=1)

# Calculating Radiative Forcing
The weighted refractive index and weighted diameter can now be used to find the radiative forcing efficiency of the aerosols.

## Final Cleaning
SSA is calculated from Abs532_stdPT_MOORE and drySC550_stdPT_MOORE. Observations without complete compostion, size, or optical data are removed.

In [35]:
# remove data without refractive index, size or SSA variables
df_new_under2k = df_new_under2k.dropna(subset=[" Abs532_stdPT_MOORE","rel_av"])

# calculate SSA from scattering and absorption
df_new_under2k["SSA"] = df_new_under2k[" drySC550_stdPT_MOORE"]/(df_new_under2k[" drySC550_stdPT_MOORE"] + df_new_under2k[" Abs532_stdPT_MOORE"])

## Calculations
The following code chunck performs 5 crutial steps:

1. Use the weighted diameter to calculate size parameter.
The mie code needs the size parameter. The size parameter ($x$) is calculated with diameter ($d$) and $\lambda$, which for us is 550 nm. The diameter and $\lambda$ must be in the same units.

$$
x = \frac{d\pi}{\lambda} 
$$

2. We use [miepython](https://miepython.readthedocs.io/en/latest/index.html) which takes refractive index and size parameter as variables. The code outputs 3 dimensionless efficiencies, but we are the most interested in the back-scattering efficiency ($Q_{back}$) and the scattering efficiency ($Q_{sca}$).

3. We use the $Q_{back}$ and the $Q_{sca}$ values to find the hemispheric backscatter fraction ($b$). 

$$
b = \frac{Q_{back}}{Q_{sca}\times 4 \pi}
$$

4. We find the average upscatter fraction ($\beta$) using the hemispheric backscatter fraction. We utalize the following equation by [Anderson 1999](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/1999JD900172).

$$
\beta = 0.082 + 1.85b -2.97b^2
$$

5. Relative radiative forcing is found with the following equation from [Langridge 2012](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2011JD017116). Our constants are the solar constant ($S_0$), atmospheric transmission ($T$), cloud fraction ($A_C$), and surface albedo ($R$). We are using the same constant values as in the paper.

$$
\Delta F_{eff} = -0.5 S_0 T^2 (1-A_C) SSA \beta \times \{(1-R)^2 - (\frac{2R}{\beta})[(\frac{1}{SSA})-1]\}
$$

In [36]:
# removes chained argument warnings
warnings.filterwarnings('ignore')

# create upscatter fraction function
# function from Alyson Fritzmann
def back_scat(i):
    beta = .082 + 1.85*i -2.97*(i**2)
    return beta

# set constants for equation, based on those from Langridge 2012
S_o = 1366
T = .76
A_c = 0
R = 0.14

# create radiative forcing function
# function from Alyson Fritzmann
def rad_forcing(SSA,BETA):
    Feff = -0.5*S_o*(T**2)*SSA*BETA*(1-A_c)*((1-R)**2 - (((2*R)/BETA)*((1/SSA) - 1)))
    return Feff

# create blank variables that will get filled during loop
df_new_under2k['ref'] = ""
df_new_under2k['rad'] = ""
df_new_under2k['sizeparam'] = ""
df_new_under2k['backscatter_frac'] = ""
df_new_under2k['upscatter_frac'] = ""

# loop for radiative forcing
for index, row in df_new_under2k.iterrows():
    # create variables for refractive index components, diameter, and lambda
    rel = row['rel_av']
    im = row['im_av']
    diam = row['LAS_weighted']
    lam = 550
    # mie python does not assume imaginary part is negative, need to add it
    m = complex(rel,-im)
    df_new_under2k['ref'][index] = m
    # STEP 1: create size parameter
    # sphere size parameter is dimensionless, radius and lambda should be in nm
    x = (diam*np.pi)/lam
    df_new_under2k['sizeparam'][index] = x
    # STEP 2: use mie code to find qback and qsca
    qext, qsca, qback, g = miepython.mie(m,x)
    # STEP 3: find backscatter fraction
    b = (qback/qsca)/(4*np.pi)
    df_new_under2k['backscatter_frac'][index] = b
    # STEP 4: find upscatter fraction
    beta = back_scat(b)
    df_new_under2k['upscatter_frac'][index] = beta
    # create variable for SSA
    SSA = row['SSA']
    # STEP 5: find relative radiative forcing
    df_new_under2k['rad'][index] = rad_forcing(SSA,beta)

## Export Data
We create new dataframe with only the columns we want for statistical analysis. This is saved as a .csv.

In [37]:
# select columns
clean_2019 = df_new_under2k[['datetime_start','year', ' Longitude',' Latitude','Org_frac','NH42SO4_frac','NH4NO3_frac','BC_frac','ref','sizeparam','backscatter_frac','upscatter_frac','SSA','rad']]
# save as .csv
clean_2019.to_csv('cleaned_data/clean_2019.csv', index=False)